# Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Install


In [ ]:
!pip install datasets==2.20.0 # transformers==4.41.2 다운그레이드 시 Trainer import 호환 오류

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.5.0 which is incompatible.


# 4.1.2 모델 초기화 후 학습

In [ ]:
from datasets import load_dataset
from transformers import BertTokenizerFast

dataset = load_dataset("klue", "ynat")
model_name = "drive/MyDrive/MyTokenizer"
tokenizer = BertTokenizerFast.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/45678 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9107 [00:00<?, ? examples/s]

In [ ]:
from transformers import BertConfig

cfg = BertConfig
print(cfg)

<class 'transformers.models.bert.configuration_bert.BertConfig'>


In [ ]:
mycfg = BertConfig(vocab_size=tokenizer.vocab_size)

In [ ]:
from transformers import BertForMaskedLM

model = BertForMaskedLM(mycfg)
print(model.config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.41.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 24000
}



In [ ]:
!pip install accelerate -U # 런타임 > 세션 다시 시작

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

datasets = dataset.map(
    lambda x: tokenizer(x["title"]),
    batched=True,
    batch_size=1000,
    remove_columns=dataset.column_names["train"],
)

args = TrainingArguments(
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    max_steps=1000,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=100,
    logging_dir="drive/MyDrive/logs",
    output_dir="drive/MyDrive/ckpt",
    report_to="tensorboard",
)

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=datasets["train"],
    eval_dataset=datasets["validation"],
    tokenizer=tokenizer,
    data_collator=collator,
)

trainer.train()
trainer.save_model("drive/MyDrive/MyBertModel")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss
100,9.720800,9.564934
200,9.373200,9.505132
300,9.321800,9.483682
400,9.284700,9.441111
500,9.228500,9.406835
600,9.176500,9.437104
700,9.190700,9.396841
800,9.226300,9.392704
900,9.194000,9.410888
1000,9.154500,9.367720
